# Loss function visualisations


### Setup


In [ ]:
%load_ext autoreload

import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch
from numpy.random import default_rng
from tqdm.notebook import tqdm
from ipywidgets import interact
import gc

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path
import einops

import ifcopenshell
import open3d as o3d

from src.elements import *
from src.preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.chamfer import *
from src.utils import *
from src.completion_vis import *
import transforms3d


This notebook contains additional experiments on loss functions.

Specifically it contains;

1. Visualising point-wise distances from a loss function
2. visualising results form point cloud reconstruction and point cloud completion methods

#### data loading and pre-processing

All data from sphere morphing must be loaded for analysis.


#### Visusalise correpsondences


In [ ]:
# visaulise losses

%autoreload 2

# load clouds
cld1_name = "output/elbow/test/24102.pcd"
cld2_name = "output/elbow/test/24106.pcd"

cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)

In [ ]:
# measure pairing loss
src = torch.Tensor([cld1, cld1])
tgt = torch.Tensor([cld2, cld2])

chamferDist = ChamferDistance()
nn = chamferDist(
    src, tgt, bidirectional=True, return_nn=True, k=1
)
dist = torch.sum(nn[0].dists) + torch.sum(nn[1].dists)

# compute the cyclical index (closest point of closest point). this should ideally be 0->n_points in order
perfect_idx = torch.range(0,nn[0].idx.shape[1]-1, dtype=int) # 0-> N_points
perfect_idx = perfect_idx[:,None] # add extra dimension
perfect_idx = perfect_idx.repeat(nn[0].idx.shape[0], 1, 1) # batch_size
true_idx_fwd = torch.gather(nn[0].idx, 1, nn[1].idx) # tgt[[src[match]]]
true_idx_bwd = torch.gather(nn[1].idx, 1, nn[0].idx) # tgt[[src[match]]]
pair_loss = torch.sum(perfect_idx == true_idx_fwd)

#print(nn[0].knn.shape, true_idx.shape, torch.flatten(true_idx[0]).shape)
#pairs = torch.gather(nn[1].knn, 1, true_idx)
paired_points_fwd = torch.stack([nn[0].knn[i][torch.flatten(nn[1].idx[i])] for i in range(nn[1].idx.shape[0])])
#paired_points_fwd = torch.stack([nn[1].knn[i][torch.flatten(true_idx_fwd[i])] for i in range(true_idx_fwd.shape[0])])
#paired_points_fwd = torch.stack([tgt[i][torch.flatten(true_idx_fwd[i])] for i in range(true_idx_fwd.shape[0])])
paired_points_fwd = paired_points_fwd.reshape((paired_points_fwd.shape[0], 
                                   paired_points_fwd.shape[1], 
                                   paired_points_fwd.shape[3])) 
pair_dist_fwd = torch.sum(torch.square(paired_points_fwd - tgt))


print("DS", true_idx_bwd.shape, nn[0].knn.shape, src.shape, nn[0].idx.shape)
paired_points_bwd = torch.stack([nn[1].knn[i][torch.flatten(nn[0].idx[i])] for i in range(nn[0].idx.shape[0])])
#paired_points_bwd = torch.stack([src[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
print(paired_points_bwd.shape)
#paired_points_bwd = torch.stack([nn[0].knn[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
paired_points_bwd = paired_points_bwd.reshape((paired_points_bwd.shape[0], 
                                   paired_points_bwd.shape[1], 
                                   paired_points_bwd.shape[3])) 
pair_dist_bwd = torch.sum(torch.square(paired_points_bwd - src))

print("pair", true_idx_bwd[0].flatten().shape)
print(dist, pair_loss, pair_dist_fwd, pair_dist_bwd)




In [ ]:
# visualise cyclical pairs
v = visualise_loss(cld1, cld2, blueprint, pairs=true_idx_bwd[0].flatten(), loss="pair", same_cloud=True)
v

In [ ]:
true_idx_fwd = torch.gather(nn[0].idx, 1, nn[1].idx) # tgt[[src[match]]]
true_idx_bwd = torch.gather(nn[1].idx, 1, nn[0].idx) # tgt[[src[match]]]

# manual chamfer loss
paired_points_bwd = torch.stack([tgt[i][torch.flatten(nn[0].idx[i])] for i in range(nn[0].idx.shape[0])])
pair_dist_bwd = paired_points_bwd - src
#pair_dist_bwd = torch.sum(torch.square(paired_points_bwd - x))
#paired_points_fwd = torch.stack([x[i][torch.flatten(nn[1].idx[i])] for i in range(nn[1].idx.shape[0])])
paired_points_fwd = torch.stack([src[i][torch.flatten(true_idx_bwd[i])] for i in range(true_idx_bwd.shape[0])])
pair_dist_fwd = paired_points_fwd - paired_points_bwd

pair_dist = pair_dist_bwd + pair_dist_fwd
pair_dist = torch.mul(torch.square(pair_dist), torch.square(pair_dist_bwd))
pair_dist = torch.sum(pair_dist) 
print(pair_dist, dist)

In [ ]:
# visualise EMD
src = torch.Tensor([cld1]).cuda()
tgt = torch.Tensor([cld2]).cuda()
loss, ass = calc_emd(src, tgt, 0.05, 1000)

ass = ass.detach().cpu().numpy()
unique = np.unique(ass)
print("unique", len(unique))

v = visualise_loss(cld1, cld2, blueprint, pairs=ass.flatten(), loss="pair", same_cloud=False)
v

In [ ]:
# visualise regular chamfer loss
v = visualise_loss(cld1, cld2, blueprint)
v

In [ ]:
# visualise reverse weighted chamfer loss
src = torch.Tensor([cld1]).cuda()
tgt = torch.Tensor([cld2]).cuda()
loss, ass = calc_reverse_weighted_cd_tensor(src, tgt, k=32, return_assignment=True)

ass = ass[0].detach().cpu().numpy()

unique = np.unique(ass)
print("unique", len(unique))

v = visualise_loss(cld1, cld2, blueprint, pairs=ass, loss="pair", same_cloud=False)
v

In [ ]:
# load clouds
cld1_name = "data/24102s.pcd"
cld2_name = "data/24103s.pcd"

cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)

In [ ]:
# visualise chamfer loss with coplanarity
target_pcd_tensor = torch.tensor([cld2], device=cuda)
src_pcd_tensor = torch.tensor([cld1], device=cuda)

vect, dists = knn_vectors(src_pcd_tensor, target_pcd_tensor, 3)
coplanarity = check_coplanarity(vect)
coplanarity = coplanarity[0].detach().cpu().numpy()
print("shapes", coplanarity.shape, dists.shape)

v = visualise_loss(cld1, cld2, blueprint, strength=coplanarity, k=3)
v

In [ ]:
print(label_list[1])

In [ ]:
# visaulise direct correspondence loss

%autoreload 2

v = visualise_parameter_pair(predictions_list, label_list, cat, blueprint, 4)
v

In [ ]:
v_s = [visualise_parameter_pair(predictions_list, label_list, cat, blueprint, i) for i in range(20)]
v_s

#### visualise cross section correspondence


In [ ]:
def draw_o3d_correspondences(a, b, ass):
    points = np.vstack([a,b])
    start_idx = np.arange(a.shape[0])
    end_idx = ass + a.shape[0]
    lines = np.stack([start_idx, end_idx], axis=1)
    print(points.shape, lines.shape, lines[:3])
    
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    return line_set

In [ ]:
cloud_path = "sphere/plane_slice2.pcd"
points = np.array(o3d.io.read_point_cloud(cloud_path).points)
# flatten
points[:,1] = 0
gt = o3d.geometry.PointCloud()
gt.points = o3d.utility.Vector3dVector(points)


In [ ]:
# generate points with some random variations
source = o3d.geometry.PointCloud()

points2 = np.copy(points)
variation = (np.random.rand(points2.shape[0], points2.shape[1]) - 0.5)/10
variation[:,1] = 0
points2 += variation
source.points = o3d.utility.Vector3dVector(points2)


In [ ]:
# # get chamfer correspondences
# chamferDist = ChamferDistance()

# nn = chamferDist(
#     x, y, bidirectional=True, return_nn=True)
# assignment = nn[0].idx[:,:,0][0].cpu().numpy()
# print(assignment.shape)

In [ ]:
# # get balanced correspondences
# chamferDist = ChamferDistance()
# eps = 0.00001

# k = 16
# k2 = 8 # reduce k to check density in smaller patches
# power = 1

# # add a loss term for mismatched pairs
# nn = chamferDist(
#     x, y, bidirectional=True, return_nn=True, k=k
# )

# # measure density with itself
# nn_x = chamferDist(x, x, bidirectional=False, return_nn=True, k=k2)
# density_x = torch.mean(nn_x[0].dists[:,:,1:], dim=2)
# density_x = 1 / (density_x + eps)
# high, low = torch.max(density_x), torch.min(density_x)
# diff = high - low
# density_x = (density_x - low) / diff

# # measure density with other cloud
# density_xy = torch.mean(nn[0].dists[:,:,:k2-1], dim=2)
# density_xy = 1 / (density_xy + eps)
# high, low = torch.max(density_xy), torch.min(density_xy)
# diff = high - low
# density_xy = (density_xy - low) / diff
# w_x = torch.div(density_xy, density_x)
# #print("w", w_x.shape, w_x[0])
# w_x = torch.pow(w_x, power)
# scaling_factors_1 = w_x.unsqueeze(2).repeat(1, 1, k)
# multiplier = torch.gather(scaling_factors_1, 1, nn[1].idx)

# scaled_dist_1 = torch.mul(nn[1].dists, multiplier)
# scaled_dist_1x, i1 = torch.min(scaled_dist_1, 2)

# # measure density with itself
# nn_y = chamferDist(y, y, bidirectional=False, return_nn=True, k=k2)
# density_y = torch.mean(nn_y[0].dists[:,:,1:], dim=2)
# density_y = 1 / (density_y + eps)
# high, low = torch.max(density_y), torch.min(density_y)
# diff = high - low
# density_y = (density_y - low) / diff

# # measure density with other cloud
# density_yx = torch.mean(nn[1].dists[:,:,:k2-1], dim=2)
# density_yx = 1 / (density_yx + eps)
# high, low = torch.max(density_yx), torch.min(density_yx)
# diff = high - low
# density_yx = (density_yx - low) / diff
# w_y = torch.div(density_yx, density_y)
# #print("w", w_x.shape, w_x[0])
# w_y = torch.pow(w_y, power)
# scaling_factors_0 = w_y.unsqueeze(2).repeat(1, 1, k)
# multiplier = torch.gather(scaling_factors_0, 1, nn[0].idx)

# scaled_dist_0 = torch.mul(nn[0].dists, multiplier)
# scaled_dist_0x, i0 = torch.min(scaled_dist_0, 2)

# min_ind_0 = torch.gather(nn[0].idx, 2, i0.unsqueeze(2).repeat(1,1,k))[:, :, 0]
# min_ind_1 = torch.gather(nn[1].idx, 2, i1.unsqueeze(2).repeat(1,1,k))[:, :, 0]

# assignment = min_ind_0[0].cpu().numpy()
# print(assignment.shape)


In [ ]:
def calc_direct(x, y):
    return torch.sum(torch.square(x-y))

In [ ]:
%autoreload 2
# morph a sphere into the shape of an input point cloud
# by optimising chamfer loss iteratively
# total points = num_points**2
def optimise_shape(src_pcd_tensor, tgt_pcd_tensor, iterations=5, learning_rate=0.01, loss_func= "chamfer"):
    
    cuda = torch.device("cuda")
    
    # optimise
    optimizer = torch.optim.Adam([tgt_pcd_tensor], lr=learning_rate)
    intermediate, losses, assingments = [], [], []
    chamferDist = ChamferDistance()
    assignments = []

    for i in tqdm(range(iterations)):
        optimizer.zero_grad()
        
        if loss_func == "chamfer":
            nn = chamferDist(
                src_pcd_tensor, tgt_pcd_tensor, bidirectional=True, return_nn=True)
            loss = torch.sum(nn[1].dists) + torch.sum(nn[0].dists)
            assignment = [nn[0].idx[:,:,0].detach().cpu().numpy(), nn[1].idx[:,:,0].detach().cpu().numpy()]
        elif loss_func == "emd":
            loss, assignment = calc_emd(tgt_pcd_tensor, src_pcd_tensor, 0.05, 50)
            assignment = assignment.detach().cpu().numpy()
        elif loss_func == "balanced":
            loss, assignment = calc_balanced_chamfer_loss_tensor(src_pcd_tensor, tgt_pcd_tensor, return_assignment=True, k=4)
        elif loss_func == "infocd":
            loss, assignment = calc_cd_like_InfoV2(src_pcd_tensor, tgt_pcd_tensor, return_assignment=True)
        elif loss_func == "direct":
            loss = calc_direct(src_pcd_tensor, tgt_pcd_tensor)
        else:
            print("unspecified loss")
            
        #print("a", assignment[0].shape)
        loss.backward()
        optimizer.step()
        print("iteration", i, "loss", loss.item())
        
        intermediate.append(tgt_pcd_tensor.clone())

    # calculate final chamfer loss
    dist = chamferDist(src_pcd_tensor, tgt_pcd_tensor, bidirectional=True)
    emd_loss, _ = calc_emd(tgt_pcd_tensor, src_pcd_tensor, 0.05, 50)
    print("final chamfer dist", dist.item(), "emd", emd_loss.item())
    
    # save assignments for analysis
#     if measure_consistency:
#         with open("sphere/assignments_" + loss_func + ".pkl", "wb") as f:
#             pickle.dump(assingments, f)
            
    intermediate = torch.stack(intermediate)
    return intermediate

In [ ]:
# create tensors
cuda = torch.device("cuda")
x = torch.tensor([points], device=cuda)
y = torch.tensor([points2], device=cuda, requires_grad=True)

iterations = 50
intermediate = optimise_shape(x, y, iterations=iterations, learning_rate=0.01, loss_func="chamfer")
print(intermediate.shape)
intermediate = intermediate.reshape((iterations, x.shape[1], x.shape[2])).detach().cpu().numpy()
print(intermediate.shape)


In [ ]:
def draw_o3d_paths(a, stops):
    steps = stops.shape[0]
    print(stops.shape)
    points = np.vstack([a]+[stops[i] for i in range(steps)])
    line_sets = []
    #print(points.shape)
    for j in  range(steps):
        if j ==0:
            start_idx = np.arange(a.shape[0])
        else:
            start_idx = start_idx + a.shape[0]
        end_idx = start_idx + a.shape[0]
        lines = np.stack([start_idx, end_idx], axis=1)
        #print(points.shape, lines.shape, lines[:3])

        colors = [[1, 1-0.1*j, 0] for i in range(len(lines))]
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
        
        line_sets.append(line_set)
    
    return line_sets

In [ ]:
interm = o3d.geometry.PointCloud()
interm.points = o3d.utility.Vector3dVector(intermediate[0])

end = o3d.geometry.PointCloud()
end.points = o3d.utility.Vector3dVector(intermediate[-1])


source.paint_uniform_color([0., 0.706, 1])
end.paint_uniform_color([0., 0.706, 1])
interm.paint_uniform_color([0.5, 0.5, 0])
gt.paint_uniform_color([1., 0.706, 1])

#line_sets = draw_o3d_paths(points2, intermediate[1:])
#o3d.visualization.draw_geometries([gt, source, interm] + line_sets)

# start from the 2nd step onwards
line_sets = draw_o3d_paths(points2, intermediate)
#o3d.visualization.draw_geometries([source, gt, end] + line_sets)
#o3d.visualization.draw_geometries([source, gt] )
o3d.visualization.draw_geometries([end, gt] )
#o3d.visualization.draw_geometries([interm, gt, end] )


In [ ]:
source.paint_uniform_color([0., 0.706, 1])
gt.paint_uniform_color([1., 0.706, 1])
gt.paint_uniform_color([1., 0.706, 1])

line_set = draw_o3d_correspondences(points2, points, assignment)
o3d.visualization.draw_geometries([gt, source, line_set])

#### Evaluate autoencoder results


In [ ]:
# visualise npy point cloud set

# chamfer
ch_savepath = "/home/haritha/documents/experiments/PointSWD/logs2/reconstruction/model/modelnet40/"
ch_inp_list = np.load(os.path.join(ch_savepath, "input.npy"))
ch_rec_list = np.load(os.path.join(ch_savepath, "reconstruction.npy"))

# new
savepath = "/home/haritha/documents/experiments/PointSWD/logs24/reconstruction/model/modelnet40/"
inp_list = np.load(os.path.join(savepath, "input.npy"))
rec_list = np.load(os.path.join(savepath, "reconstruction.npy"))

print(rec_list.shape) 

In [ ]:
blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)
limit = 10
vis = []

for i in range(limit):
    vis.append(vis_ifc_and_cloud(ifc, [ch_inp_list[i].astype("float64"), ch_rec_list[i].astype("float64")]))

vis

In [ ]:
blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)
limit = 10
vis = []

for i in range(limit):
    vis.append(vis_ifc_and_cloud(ifc, [inp_list[i].astype("float64"), rec_list[i].astype("float64")]))

vis

In [ ]:
# compare losses
def evaluate_autoencoder_results(inp_list, rec_list, loss_funcs, batch_size=None):
    cuda = torch.device("cuda")
    if batch_size == None:
        batch_size = len(inp_list)
        
    # create empty dict
    all_losses = {func:[] for func in loss_funcs}
    
    # split into batches
    for i in tqdm(range(math.ceil(len(inp_list)/batch_size))):
        a = i*batch_size
        b = min(a+batch_size, len(inp_list))
        #print(a,b)
        
        # compute losses
        inp_tensor = torch.tensor(inp_list[a:b], device=cuda)
        rec_tensor = torch.tensor(rec_list[a:b], device=cuda)
        losses = calculate_3d_loss(inp_tensor, rec_tensor, loss_funcs)
        
        # sum losses
        for k, v in losses.items():
            all_losses[k].append(v)
            
    # average losses
    for k in all_losses:
        all_losses[k] = np.average(np.array(all_losses[k]))
    return all_losses

In [ ]:
%autoreload 2

loss_funcs = ["chamfer", "reverse", "emd"]

# chamfer
ch_loss = evaluate_autoencoder_results(ch_inp_list, ch_rec_list, loss_funcs, 512)

# new
loss = evaluate_autoencoder_results(inp_list, rec_list, loss_funcs, 512)

print("chamfer tuned", ch_loss)
print("new", loss)

### completion evaluation


#### DCD (VCN plus, MVP) / PointAttn (PCN)


In [ ]:
#DCD(VCN)
balanced_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs_balanced/"
dcd_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs_dcd/"

#pointAttN
# balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs/"
# dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"

cld_balanced = get_cloud_list_vcn(balanced_path, "pred")
cld_dcd = get_cloud_list_vcn(dcd_path, "pred")
cld_gt = get_cloud_list_vcn(dcd_path, "gt")
cld_partial = get_cloud_list_vcn(dcd_path, "partial")

d_balanced = get_density(cld_balanced)
d_dcd = get_density(cld_dcd)
d_gt = get_density(cld_gt)
d_partial = get_density(cld_partial)

print(d_partial.shape, d_gt.shape)

# produce density colourmaps that are normalised with their pairs
d_balanced, d_dcd, d_gt, d_partial = normalise_densities([d_balanced, d_dcd, d_gt, d_partial])

v_balanced, _ = get_coloured_clouds(cld_balanced, d_balanced)
v_dcd, _ = get_coloured_clouds(cld_dcd, d_dcd)
v_gt, col = get_coloured_clouds(cld_gt, d_gt)
v_partial, _ = get_coloured_clouds(cld_partial, d_partial)
# c_dcd = get_colours(d_dcd)
# c_gt = get_colours(d_gt)
# c_partial = get_colours(d_partial)



In [ ]:
print(d_partial.shape, d_gt.shape)


In [ ]:
print(len(v_balanced))
#shortlist = [226,227,228,102,103,104,0,1,2] = [39,140,219,106,1,7,148,156,11,43,53,194,4,17,91,44,66,3,27,274,87,55,158,35,103,129,112,170,195]
#shortlist = [39,140,7,148,53,194,91,274,55,158,129,112,195]
for i in range(100,150):
#for i in shortlist:
    print(i)
    view_side_by_side(v_partial, v_dcd, v_balanced, v_gt, i+1)


In [ ]:
# save a desired set of clouds
def save_cloud(clouds,  name, i, colours=None):
    directory = "mitsuba/"
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(clouds[i])
#     o3d.io.write_point_cloud(directory + name + str(i) + ".ply", pcd)
    with open(directory + name + str(i) + ".npy", "wb") as f:
        np.save(f, clouds[i])
    if colours is not None:
        with open(directory + name + str(i) + "c_.npy", "wb") as f:
            np.save(f, colours[i])

i = 4
# save_cloud(cld_balanced, "balanced", i)
# save_cloud(cld_dcd, "dcd", i)
# save_cloud(cld_partial, "partial", i)
# save_cloud(cld_gt, "gt", i)

for i in shortlist:
    save_cloud(cld_gt, "gt", i+1)
    save_cloud(cld_balanced, "bal", i+1)
    save_cloud(cld_dcd, "dcd", i+1)
    save_cloud(cld_partial, "part", i+1)



In [ ]:
balanced_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs_balanced/"
dcd_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs_dcd/"
blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)

v_balanced = view_cloud_list_vcn(balanced_path, "pred", ifc, 0)
v_dcd = view_cloud_list_vcn(dcd_path, "pred", ifc, 1)
v_gt = view_cloud_list_vcn(dcd_path, "gt", ifc, 2)
v_partial = view_cloud_list_vcn(dcd_path, "partial", ifc, 2)

In [ ]:
print(len(v_balanced))
for i in range(len(v_balanced)):
    print(v_balanced[i], v_dcd[i], v_gt[i], v_partial[i])

In [ ]:
o3d.visualization.draw_geometries([point_cloud, gt])

#### InfoCD (Seedformer, PCN)


In [ ]:
%autoreload 2


# load NPYs from each. (GT, Complete, CD, InfoCD, UniformCD)
category_id = "02933112/"
cd_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_22_14_49_07/outputs_cd/"
info_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_22_14_49_07/outputs_info/"
uniform_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/pcn/train_pcn_Log_2024_02_29_23_43_45/outputs_uniform/"

blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)


In [ ]:
v_cd = view_cloud_list_seedformer(category_id, cd_path, "pred", ifc, 0)
v_info = view_cloud_list_seedformer(category_id, info_path, "pred", ifc, 1)
v_uniform = view_cloud_list_seedformer(category_id, uniform_path, "pred", ifc, 2)
v_complete = view_cloud_list_seedformer(category_id, cd_path, "gt", ifc, 0)


In [ ]:
seedformer_output_path = "/home/haritha/documents/experiments/seedformer_clone/ICCV2023-HyperCD/output_vis/bbox/02691156/"

view_seedformer_outputs(seedformer_output_path)

In [ ]:
# visualise completion results and their bboxes
#DCD(VCN)
#balanced_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs/"
# dcd_path = "/home/haritha/documents/experiments/Density_aware_Chamfer_Distance/outputs_dcd/"

#pointAttN
balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs/"
#balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_corrected_bbox/"
#dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"

cld_completed = get_cloud_list_vcn(balanced_path, "pred")
cld_gt = get_cloud_list_vcn(balanced_path, "gt")
cld_partial = get_cloud_list_vcn(balanced_path, "partial")
cld_partial = cld_partial.transpose(0, 2, 1)

print(cld_partial.shape)

completed_bboxes = get_bboxes(cld_completed)
gt_bboxes = get_bboxes(cld_gt)
partial_bboxes = get_bboxes(cld_partial)

limit = 15
#visualize_point_clouds_with_bboxes_and_cameras(cld_completed[:limit], cld_partial[:limit], cld_gt[:limit], completed_bboxes, gt_bboxes, partial_bboxes)


In [ ]:
visualize_point_clouds_with_bboxes_and_cameras(cld_completed[:limit], cld_partial[:limit], cld_gt[:limit], completed_bboxes, gt_bboxes, partial_bboxes)


In [ ]:
# calculate and filter by bounding box overlap between ground truth and partial cloud
def calc_bbox_overlap(output, gt):
    pred_min_values, _ = torch.min(output, dim=1)
    pred_max_values, _ = torch.max(output, dim=1)
    gt_min_values, _ = torch.min(gt, dim=1)
    gt_max_values, _ = torch.max(gt, dim=1)

    intersection_min = torch.max(pred_min_values, gt_min_values)
    intersection_max = torch.min(pred_max_values, gt_max_values)
    intersection_dims = torch.clamp(intersection_max - intersection_min, min=0)
    intersection_volume = torch.prod(intersection_dims, dim=-1)

    pred_volume = torch.prod(pred_max_values - pred_min_values, dim=-1)
    gt_volume = torch.prod(gt_max_values - gt_min_values, dim=-1)
    union_volume = pred_volume + gt_volume - intersection_volume
    iou = intersection_volume / union_volume
    return iou


In [ ]:

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)


cld_partial_t = torch.tensor(cld_partial).cuda()
cld_gt_t = torch.tensor(cld_gt).cuda()
overlap = calc_bbox_overlap(cld_partial_t, cld_gt_t)
print(overlap.shape, overlap.max(), overlap.min(), overlap.mean())

threshold = 0.8
above_threshold = overlap > threshold
print(above_threshold.sum().item())

# plot the distribution of bbox overlaps
plt.hist(overlap.cpu().numpy(), bins=50, color='#72195a')

In [ ]:
# visualise bbox only (frozen) predictions (pointattN)
data_path = "/home/haritha/documents/experiments/PointAttN/outputs_frozen_bbox/"

cld_completed = get_cloud_list_vcn(data_path, "pred", pred_bbox=True)
cld_gt = get_cloud_list_vcn(data_path, "gt", pred_bbox=True)
pred_bboxes = get_cloud_list_vcn(data_path, "pred_bbox", pred_bbox=True)
cld_partial = get_cloud_list_vcn(data_path, "partial", pred_bbox=True)
cld_partial = cld_partial.transpose(0, 2, 1)

print(cld_partial.shape)

gt_bboxes = get_bboxes(cld_gt)
print("gt bbox", gt_bboxes.shape)
completed_bboxes = convert_bboxes_to_min_max(pred_bboxes)


limit = 5
visualize_point_clouds_with_bboxes_and_cameras(cld_completed[:limit], cld_partial[:limit], cld_gt[:limit], completed_bboxes, gt_bboxes)



### visualise camera


In [ ]:
# visualise camera predictions (pointattN)
#data_path = "/home/haritha/documents/experiments/PointAttN/outputs_frozen_camera/"
data_path = "/home/haritha/documents/experiments/PointAttN/outputs_camera_original/"
data_path_ucd = "/home/haritha/documents/experiments/PointAttN/outputs_camera_ucd/"
data_path_cd = "/home/haritha/documents/experiments/PointAttN/outputs_camera_cd/"

limit = 100

pred_camera_cd = get_cloud_list_vcn(data_path_cd, "pred_camera", pred_camera=True, limit=limit)
pred_camera_ucd = get_cloud_list_vcn(data_path_ucd, "pred_camera", pred_camera=True, limit=limit)
pred_camera_original = get_cloud_list_vcn(data_path, "pred_camera", pred_camera=True, limit=limit)

cld_completed = get_cloud_list_vcn(data_path, "pred", pred_camera=True, limit=limit)
cld_gt = get_cloud_list_vcn(data_path, "gt", pred_camera=True, limit=limit)
gt_camera = get_cloud_list_vcn(data_path, "gt_camera", pred_camera=True, limit=limit)
cld_partial = get_cloud_list_vcn(data_path, "partial", pred_camera=True, limit=limit)
cld_partial = cld_partial.transpose(0, 2, 1)

#print(gt_camera.shape)

#print("gt", gt_camera, "pred", pred_camera_ucd)
visualize_point_clouds_with_bboxes_and_cameras(cld_gt[:limit], cld_partial[:limit],
                                  cameras1=pred_camera_original[:limit], cameras2=pred_camera_cd[:limit],
                                  cameras3=pred_camera_ucd[:limit], cameras4=gt_camera[:limit])
# visualize_point_clouds_with_bboxes_and_cameras(cld_completed[:limit], cld_partial[:limit], cld_gt[:limit],
#                                  cameras1=pred_camera[:limit], cameras2=gt_camera[:limit])


In [ ]:
pose_dir = "/home/haritha/documents/experiments/pcn/pcn_camera/test/pose/"
pcd_dir = "/home/haritha/documents/experiments/pcn/pcn_camera/test/pcd/"
file_name = "02958343/1b23106e1c447efc36c8c5d8a85eb4f9/0"

# load camera parameters and point cloud
with open(pose_dir + file_name + ".txt", "rb") as f:
    pose = np.loadtxt(f)
    location, direction = extract_camera_location_and_direction(pose)

print(pose)
pcd = o3d.io.read_point_cloud(pcd_dir + file_name + ".pcd")
pcd = np.asarray(pcd.points)
# visualise
visualize_point_cloud_with_camera_axis(pcd, location, direction)


#visualize_point_cloud_with_camera_cone(pcd, gt_camera[0])

In [ ]:
# test that transformations for the camera are correct

# perform transformations
rnd_value = np.random.uniform(0, 1)

trfm_mat = transforms3d.zooms.zfdir2mat(1)
trfm_mat_x = np.dot(transforms3d.zooms.zfdir2mat(-1, [1, 0, 0]), trfm_mat)
trfm_mat_z = np.dot(transforms3d.zooms.zfdir2mat(-1, [0, 0, 1]), trfm_mat)
if rnd_value <= 0.25:
    trfm_mat = np.dot(trfm_mat_x, trfm_mat)
    trfm_mat = np.dot(trfm_mat_z, trfm_mat)
elif rnd_value > 0.25 and rnd_value <= 0.5:
    trfm_mat = np.dot(trfm_mat_x, trfm_mat)
elif rnd_value > 0.5 and rnd_value <= 0.75:
    trfm_mat = np.dot(trfm_mat_z, trfm_mat)

# Apply transformation to point clouds
pcd_tr = np.dot(pcd, trfm_mat.T)

# Transform camera position and axis
transformed_camera_position = np.dot(location, trfm_mat.T)
transformed_camera_axis = np.dot(direction, trfm_mat.T)
transformed_camera_axis = transformed_camera_axis / np.linalg.norm(transformed_camera_axis)  # Ensure it's a unit vector

# visualise results
visualize_point_cloud_with_camera_axis(pcd_tr, transformed_camera_position, transformed_camera_axis)



### confidence visualisations and plotting

In [ ]:
%autoreload 2

# normalise for each example across prediction sets
def normalise_confidences(confidences, log=True):
    #print(confidences.shape)
    if log:
        confidences = np.log(confidences + 1)
    high = np.max(confidences)*0.8
    low = np.min(confidences)
    # high = 0.11
    # low = 0
    print("high", high)
    confidences = (confidences - low) / (high - low)

    return confidences


# visualise confidence predictions (pointattN)
balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_confidence/"
#balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_corrected_bbox/"
#dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"
load_idx = True
limit = 149
cld_completed = get_cloud_list_vcn(balanced_path, "pred", pred_confidence=True, limit=limit, load_idx=load_idx)
cld_gt = get_cloud_list_vcn(balanced_path, "gt", pred_confidence=True, limit=limit, load_idx=load_idx)
cld_partial = get_cloud_list_vcn(balanced_path, "partial", pred_confidence=True, limit=limit, load_idx=load_idx)
confidences = get_cloud_list_vcn(balanced_path, "confidence", pred_confidence=True, limit=limit, load_idx=load_idx)
idxs = get_cloud_list_vcn(balanced_path, "idx", pred_confidence=True, limit=limit, load_idx=load_idx)
cld_partial = cld_partial.transpose(0, 2, 1)

#print(cld_partial.shape)

completed_bboxes = get_bboxes(cld_completed)
gt_bboxes = get_bboxes(cld_gt)
partial_bboxes = get_bboxes(cld_partial)

limit = 2000

# normalise densities and colour clouds
confidences = normalise_confidences(confidences[:limit])

#v_completed, _ = get_coloured_clouds(cld_completed[:limit], confidences[:limit])

# visualize_point_clouds_with_bboxes_and_cameras(v_completed[:limit], cld_partial[:limit], cld_gt[:limit],
#                                                completed_bboxes, gt_bboxes, partial_bboxes, paint_preds=False)



In [ ]:
if load_idx:
    for i, id in enumerate(idxs):
        print(i, id)

In [ ]:
print(np.max(confidences), np.min(confidences), np.max(confidences[12]), np.min(confidences[12]))

#### plot confidence

In [ ]:
#### confidence plotting
print(max(confidences.flatten()))
# plot the distribution of confidence values
#plt.hist(confidences.flatten(), bins=500, color='#72195a')

# compute pointwise chamfer distance
def calc_pointwise_chamfer_distance(pred, gt):
    chamferDist = ChamferDistance()
    nn = chamferDist(
        pred, gt, bidirectional=True, return_nn=True)
    return nn[0].dists

print(cld_completed.shape, cld_gt.shape)
cld_completed_t = torch.tensor(cld_completed).cuda()
cld_gt_t = torch.tensor(cld_gt).cuda()

# calculate pointwise chamfer distance
pointwise_cd = calc_pointwise_chamfer_distance(cld_completed_t, cld_gt_t).squeeze().detach().cpu().numpy()
print("pointwise cd", pointwise_cd.shape, confidences.shape)

# pointwise_cd = pointwise_cd.flatten()
# confidences = confidences.flatten()




In [ ]:
x = np.random.exponential(scale=1.0, size=1000)
y = np.random.exponential(scale=1.0, size=1000)

log_x = np.log(x + 1)  # Adding 1 to avoid log(0)
log_y = np.log(y + 1)

# Scatter plot of log-transformed data
plt.scatter(confidences, np.log(pointwise_cd+1), alpha=0.5, color='purple')
#plt.scatter(log_x, log_y, alpha=0.5, color='purple')

plt.title('Log-Transformed Scatter Plot of Two Negative Exponential Distributions')
plt.xlabel('Log of Distribution 1 (x)')
plt.ylabel('Log of Distribution 2 (y)')
plt.show()

# # Create a scatter plot to visualize correlation
# plt.scatter(x, y, alpha=0.5, color='purple')
# plt.title('Scatter Plot of Two Negative Exponential Distributions')
# plt.xlabel('Distribution 1 (x)')
# plt.ylabel('Distribution 2 (y)')
# plt.show()

# # Optionally, calculate and print the correlation coefficient
correlation = np.corrcoef(confidences, pointwise_cd)[0, 1]
print(f"Correlation coefficient: {correlation}")


In [ ]:
# Assuming X and Y are the input arrays of shape [n, m]
def split_and_calculate_means(X, Y, threshold):
    # Initialize arrays to store the means and standard deviations
    mean_set1 = np.zeros(X.shape[0])
    std_set1 = np.zeros(X.shape[0])
    mean_set2 = np.zeros(X.shape[0])
    std_set2 = np.zeros(X.shape[0])

    for i in range(X.shape[0]):
        # Apply threshold to create masks
        mask_set1 = X[i] < threshold
        mask_set2 = ~mask_set1

        # Calculate mean and std for the first subset (where X < threshold)
        if np.any(mask_set1):
            mean_set1[i] = np.mean(Y[i][mask_set1])
            std_set1[i] = np.std(Y[i][mask_set1])
        else:
            mean_set1[i] = np.nan
            std_set1[i] = np.nan

        # Calculate mean and std for the second subset (where X >= threshold)
        if np.any(mask_set2):
            mean_set2[i] = np.mean(Y[i][mask_set2])
            std_set2[i] = np.std(Y[i][mask_set2])
        else:
            mean_set2[i] = np.nan
            std_set2[i] = np.nan

    return mean_set1, std_set1, mean_set2, std_set2

# Example usage
n, m = 5, 10  # Example dimensions
X = np.random.rand(n, m)  # Random example data
Y = np.random.rand(n, m)
threshold = 0.075 # Example threshold

mean_set1, std_set1, mean_set2, std_set2  = split_and_calculate_means(confidences, pointwise_cd, threshold)
# print("Mean values for set 1 (X < threshold):", mean_set1)
# print("Mean values for set 2 (X >= threshold):", mean_set2)

print(mean_set1.shape, mean_set2.shape)
print(max(mean_set1), min(mean_set1), max(mean_set2), min(mean_set2))

In [ ]:
def sort_means(mean_set1, std_set1, mean_set2, std_set2):
    # Get the indices that would sort mean_set2 in descending order
    sorted_indices = np.argsort(mean_set2)[::-1]  # Descending order

    # Sort all means and standard deviations based on the sorted indices
    sorted_mean_set1 = mean_set1[sorted_indices]
    sorted_std_set1 = std_set1[sorted_indices]
    sorted_mean_set2 = mean_set2[sorted_indices]
    sorted_std_set2 = std_set2[sorted_indices]

    return sorted_mean_set1, sorted_std_set1, sorted_mean_set2, sorted_std_set2

# Modified plotting function to show shaded areas for standard deviations with marker size control
def plot_mean_sets_with_std(mean_set1, std_set1, mean_set2, std_set2, marker_size=8):
    n = len(mean_set1)

    plt.figure(figsize=(10, 8))

    x_values = np.arange(n)

    # Plot mean_set2 with markers only
    plt.plot(x_values, mean_set2, label='High uncertainty points', marker='o', linestyle='None', 
             color='tomato', markersize=marker_size, linewidth=2)

    # Fill between mean +/- std for set 2
    plt.fill_between(x_values, mean_set2 - std_set2/2, mean_set2 + std_set2/2, color='tomato', alpha=0.3)

    # Plot mean_set1 with markers only
    plt.plot(x_values, mean_set1, label='Low uncertainty points', marker='o', linestyle='None', 
             color='blueviolet', markersize=marker_size, linewidth=2)

    # Fill between mean +/- std for set 1
    plt.fill_between(x_values, mean_set1 - std_set1/2, mean_set1 + std_set1/2, color='blueviolet', alpha=0.4)

    # Add titles and labels
    plt.title('Chamfer distance distribution mean plot', fontsize=15)
    plt.xlabel('Point Cloud Index', fontsize=14)
    plt.ylabel('Point-wise Chamfer distance', fontsize=14)

    plt.xticks(fontsize=12)  # X-axis tick labels font size
    plt.yticks(fontsize=12)  # Y-axis tick labels font size

    # Add grid and legend
    plt.grid(True)
    plt.legend(fontsize=14)

    # Show plot
    plt.show()

# Sort the means and standard deviations based on mean_set2 in descending order
sorted_mean_set1, sorted_std_set1, sorted_mean_set2, sorted_std_set2 = sort_means(mean_set1, std_set1, mean_set2, std_set2)

# Plot the sorted means and standard deviations
plot_mean_sets_with_std(sorted_mean_set1, sorted_std_set1, sorted_mean_set2, sorted_std_set2, marker_size=2)


##### visualise confidence

In [ ]:
start = 150
limit = 175

v_completed, _ = get_coloured_clouds(cld_completed[start:limit], confidences[start:limit])

print(len(v_completed))

visualize_point_clouds_with_bboxes_and_cameras(v_completed, cld_partial[start:limit],
                                               cld_gt[start:limit], completed_bboxes[start:limit],
                                               gt_bboxes[start:limit],
                                               paint_preds=False)

#visualize_point_clouds_with_bboxes_and_cameras(v_completed, paint_preds=False)


In [ ]:
# save cloud with open io
#print(v_completed[0].points)
# gt_chair = o3d.geometry.PointCloud()
# gt_chair.points = o3d.utility.Vector3dVector(cld_partial[12])
# o3d.io.write_point_cloud("completion/cahir_gt.pcd", gt_chair)

In [ ]:
def filter_and_random_pad_point_clouds(point_clouds, masks):
    """
    Filter point clouds using masks and pad them by randomly repeating existing points 
    to the size of the largest filtered point cloud.

    Parameters:
    - point_clouds (np.ndarray): Array of shape (b, n, 3) where `b` is the batch size, 
                                 `n` is the number of points, and 3 are the coordinates (x, y, z).
    - masks (np.ndarray): Boolean array of shape (b, n) where True indicates the point is kept.

    Returns:
    - np.ndarray: The padded point clouds with uniform size within the batch.
    """
    batch_size = point_clouds.shape[0]
    filtered_point_clouds = [point_clouds[i][masks[i]] for i in range(batch_size)]
    max_points = max(len(pc) for pc in filtered_point_clouds)  # Find the maximum size

    # Pad each filtered point cloud to have `max_points` by randomly repeating points
    padded_point_clouds = []
    for pc in filtered_point_clouds:
        if len(pc) < max_points:
            # Randomly choose indices from the existing points to repeat
            indices_to_repeat = np.random.choice(len(pc), size=max_points - len(pc), replace=True)
            repeated_points = pc[indices_to_repeat]
            # Append the repeated points to the original filtered points
            padded_pc = np.vstack([pc, repeated_points])
        else:
            padded_pc = pc
        padded_point_clouds.append(padded_pc)

    return np.array(padded_point_clouds)

# clip using confidence
def clip_by_confidence(cld, confidence, threshold=0.1):
    #confidence = np.log(confidence + 1)
    #print(cld.shape, confidence)
    #high = np.max(confidence)
    high = 1.
    confidence = confidence / high
    print("high", high)
    #threshold = threshold * high
    mask = confidence < threshold
    #mask = einops.repeat(mask, 'b n -> b n c', c=3)
    print("conf", confidence.shape, np.max(confidence, axis=1))
    print("mask", mask.shape, np.sum(mask, axis=1))
    clipped_cld = filter_and_random_pad_point_clouds(cld, mask)
    #cld = cld[mask]
    print(clipped_cld.shape)
    return clipped_cld

start = 15
limit = 35
cld_completed_clipped = clip_by_confidence(cld_completed[start:limit], confidences[start:limit],
                                           threshold=0.75)

v_completed, _ = get_coloured_clouds(cld_completed_clipped, confidences[start:limit])

print(cld_completed_clipped.shape)
clipped_bboxes = get_bboxes(cld_completed_clipped)

visualize_point_clouds_with_bboxes_and_cameras(cld_completed[start:limit], cld_completed_clipped,
                                               cld_gt[start:limit], completed_bboxes[start:limit],
                                               gt_bboxes[start:limit], clipped_bboxes)


In [ ]:
#plot confidence distributions of clouds
def plot_confidence_distributions(data, num_bins=100):
    """
    Plots the distribution of confidence values for each pointset with normalized axes.

    Parameters:
    - data (np.ndarray): A numpy array of shape (batchsize, 16384) containing confidence values.
    - num_bins (int): The number of bins to use in the histogram.

    """
    batch_size = data.shape[0]
    global_min = np.min(data)
    global_max = np.max(data)

    # Pre-calculate histogram data to find global maximum frequency
    hist_data = [np.histogram(data[i], bins=num_bins, range=(global_min, global_max)) for i in range(batch_size)]
    max_frequency = max([max(hist[0]) for hist in hist_data])  # Maximum frequency across all histograms

    for i in range(batch_size):
        plt.figure(figsize=(10, 6))
        plt.hist(data[i], bins=num_bins, range=(global_min, global_max), alpha=0.75, color='blue', edgecolor='black')
        plt.title(f'Distribution of Confidence Values for Pointset {i+1}')
        plt.xlabel('Confidence Value')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.ylim(0, max_frequency + max_frequency * 0.1)  # Setting the y-axis limit slightly higher for aesthetics
        plt.show()



plot_confidence_distributions(confidences[:limit])

### kitti

In [ ]:
pointattn_kitt_path = "/home/haritha/documents/experiments/PointAttN/outputs_kitti/"

cld_completed = load_kitti_results_pointattn(pointattn_kitt_path, "pred")
bbox = load_kitti_results_pointattn(pointattn_kitt_path, "bbox")
cld_partial = load_kitti_results_pointattn(pointattn_kitt_path, "partial")

cld_partial = cld_partial.transpose(0, 2, 1)
completed_bboxes = get_bboxes(cld_completed)
gt_bboxes = get_bboxes(bbox)
#print(gt_bboxes.shape)

limit = 10
#visualize_point_clouds_with_bboxes_and_cameras(cld_completed[:limit], cld_partial[:limit], None, completed_bboxes, gt_bboxes)
visualize_point_clouds_with_bboxes_and_cameras(cld_completed[45:50], cld_partial[45:50], None, completed_bboxes[45:50], gt_bboxes[45:50])

In [ ]:
# TODO: FIX ORIENTATION, AND PICK SOME CLOUDS TO TEST ON


### IFC sign

In [ ]:
import ifcopenshell
import pywavefront
import numpy as np
import uuid
import base64

def create_ifc_guid(): return ifcopenshell.guid.compress(uuid.uuid1().hex)


def setup_ifc_file(schema_version, blueprint):
    """Setup the basic IFC file structure with a project, site, and geometric context."""

    ifc = ifcopenshell.open(blueprint)
    new_ifc = ifcopenshell.file(schema=schema_version)

    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    new_ifc.createIfcOwnerHistory()

    new_ifc.add(project)
    new_ifc.add(context)

    owner_history = new_ifc.by_type("IfcOwnerHistory")[0]
    project = new_ifc.by_type("IfcProject")[0]
    project.Name = "Road Infrastructure Project"
    context = new_ifc.by_type("IfcGeometricRepresentationContext")[0]

    # ifc_file = ifcopenshell.file(schema=schema_version)
    # owner_history = ifc_file.createIfcOwnerHistory()
    # project = ifc_file.createIfcProject(create_ifc_guid(), owner_history, Name="Road Infrastructure Project")
    site = new_ifc.createIfcSite(create_ifc_guid(), owner_history, Name="Site for Road Signs")
    new_ifc.createIfcRelAggregates(create_ifc_guid(), owner_history, RelatingObject=project, RelatedObjects=[site])

    # # Ensure CoordinateSpaceDimension is passed as an integer, not a string
    # world_coordinate_system = ifc_file.createIfcAxis2Placement3D(ifc_file.createIfcCartesianPoint([0.0, 0.0, 0.0]))
    # context = ifc_file.createIfcGeometricRepresentationContext(None, "Model", 3, 1e-5, world_coordinate_system)  # Corrected argument order and types
    # ifc_file.createIfcRelAggregates(create_ifc_guid(), owner_history, RelatingObject=project, RelatedObjects=[context])

    return new_ifc, project, site, context


def load_obj_data(obj_file):
    """Load the OBJ file and prepare vertex and index data."""
    scene = pywavefront.Wavefront(obj_file, collect_faces=True)
    f = scene.mesh_list[0].faces
    v = scene.vertices

    fx = []
    for i in f:
        fx.append([i[0]+1, i[1]+1, i[2]+1])

    return v, fx


def create_ifc_sign(ifc_file, site, context, vertices, indices):
    """Create an IfcSign using the geometric data from the OBJ file."""
    point_list = ifc_file.createIfcCartesianPointList3D(vertices)
    faces = [ifc_file.createIfcIndexedPolygonalFace(index) for index in indices]
    poly_face_set = ifc_file.createIfcPolygonalFaceSet(point_list, False, faces)
    shape_rep = ifc_file.createIfcShapeRepresentation(context, 'Body', 'Tessellation', [poly_face_set])
    product_def_shape = ifc_file.createIfcProductDefinitionShape(Representations=[shape_rep])

    direction = (0.0, 0.0, 1.0)  # Z-axis pointing up
    ref_direction = (1.0, 0.0, 0.0)  # X-axis reference

    B1_Point = ifc_file.createIfcCartesianPoint((0.0, 0.0, 0.0))
    B1_Axis2Placement = ifc_file.createIfcAxis2Placement3D(B1_Point)
    B1_Axis2Placement.Axis = ifc_file.createIfcDirection(direction)
    B1_Axis2Placement.RefDirection = ifc_file.createIfcDirection(ref_direction)
    B1_Placement = ifc_file.createIfcLocalPlacement(site.ObjectPlacement, B1_Axis2Placement)

    sign = ifc_file.createIfcSign(create_ifc_guid(), ifc_file.by_type('IfcOwnerHistory')[0], Name="Road Sign", ObjectPlacement=B1_Placement, Representation=product_def_shape)
    ifc_file.createIfcRelContainedInSpatialStructure(create_ifc_guid(), ifc_file.by_type('IfcOwnerHistory')[0], RelatedElements=[sign], RelatingStructure=site)
    return sign


def add_properties(ifc_file, sign):
    """Add custom properties to the IfcSign."""
    pset = ifc_file.createIfcPropertySet(
        GlobalId=create_ifc_guid(),
        OwnerHistory=ifc_file.by_type('IfcOwnerHistory')[0],
        Name="Pset_SignCommon",
        Description="Custom Properties for Road Sign",
        HasProperties=[
            ifc_file.createIfcPropertySingleValue("SignID", "Identifier of the Sign", ifc_file.create_entity('IfcIdentifier', ''), None),
            ifc_file.createIfcPropertySingleValue("SignChainageValue", "Chainage value of the Sign", ifc_file.create_entity('IfcText', ''), None),
            ifc_file.createIfcPropertySingleValue("Latitude", "Latitude of the Sign", ifc_file.create_entity('IfcReal', 0.), None),
            ifc_file.createIfcPropertySingleValue("Longitude", "Longitude of the Sign", ifc_file.create_entity('IfcReal', 0.), None),
            ifc_file.createIfcPropertySingleValue("Altitude", "Altitude of the Sign", ifc_file.create_entity('IfcReal', 0.), None),
            ifc_file.createIfcPropertySingleValue("BoardFace", "Visible face of the board", ifc_file.create_entity('IfcText', ''), None),
            ifc_file.createIfcPropertySingleValue("BoardBack", "Back face of the board", ifc_file.create_entity('IfcText', ''), None),
            # Add more properties as required
        ]
    )
    ifc_file.createIfcRelDefinesByProperties(create_ifc_guid(), ifc_file.by_type('IfcOwnerHistory')[0], None, None, RelatedObjects=[sign], RelatingPropertyDefinition=pset)

obj_file = "untitled222.obj"
blueprint = 'data/sample.ifc'
output_file = "output_file.ifc"
ifc_file, project, site, context = setup_ifc_file("IFC4X3_RC2", blueprint)
vertices, indices = load_obj_data(obj_file)
sign = create_ifc_sign(ifc_file, site, context, vertices, indices)
add_properties(ifc_file, sign)
ifc_file.write(output_file)
print("Generated IFC file with IfcSign from OBJ data and custom properties.")



In [ ]:
x = torch.rand(8)
y = torch.rand(8)
print(x[:-1].shape)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create a grid of x and y values
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
x, y = np.meshgrid(x, y)

# Calculate the loss value based on the given condition
threshold = 0.1
l_conditional_corrected = np.where(np.abs(x - y) < (threshold*(x+y)), (x + y + (threshold*(x+y))) / 2, np.maximum(x, y))

# Create the plot
fig = go.Figure(data=[go.Surface(z=l_conditional_corrected, x=x, y=y)])

# Update the layout with increased height
fig.update_layout(
    title='3D Surface plot of l = (x+y)/2 if |x-y| < 1 else max(x,y)',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='l'
    ),
    height=800  # Increase the height of the plot
)

# Show the plot
fig.show()


In [ ]:
# go through test set, and filter point clouds with low overlap between partial and gt
root = "/home/haritha/documents/experiments/ICCV2023-HyperCD/ShapeNetCompletion/test/"
testset_path = root + "partial/"
output_path = root + "partial_overlap/"
gt_path = root + "complete/"
save_count = 0
total = 0
class_folders = os.listdir(testset_path)

for class_f in tqdm(class_folders):
    print(class_f)
    class_path = os.path.join(testset_path, class_f)
    output_class_path = os.path.join(output_path, class_f)
    gt_class_path = os.path.join(gt_path, class_f)
    os.makedirs(output_class_path, exist_ok=True)

    obj_folders = os.listdir(class_path)
    print(len(obj_folders))

    for obj_f in obj_folders:
        total+=1
        output_obj_path = os.path.join(output_class_path, obj_f)
        gt_pcd = np.array(o3d.io.read_point_cloud(os.path.join(gt_class_path, obj_f+".pcd")).points)
        partial_pcd = np.array(o3d.io.read_point_cloud(os.path.join(class_path, obj_f, "00.pcd")).points)
        overlap = calc_bbox_overlap(torch.tensor(np.expand_dims(partial_pcd, 0)).cuda(),
                                    torch.tensor(np.expand_dims(gt_pcd, 0)).cuda())
        overlap = overlap.cpu().numpy()[0]
        #print(overlap)

        if overlap <= 0.8:
            os.makedirs(output_obj_path, exist_ok=True)
            o3d.io.write_point_cloud(os.path.join(output_obj_path, "00.pcd"),
                                    o3d.geometry.PointCloud(o3d.utility.Vector3dVector(partial_pcd)))
            save_count += 1

print(save_count, total)


### test for bijectivity

In [ ]:
%autoreload 2


# # load data (pointattN) SCD
# balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_scd/"
# #balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_corrected_bbox/"
# #dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"
# load_idx = True
# limit = 150
# cld_completed = get_cloud_list_vcn(balanced_path, "pred", limit=limit, load_idx=load_idx)
# cld_gt = get_cloud_list_vcn(balanced_path, "gt", limit=limit, load_idx=load_idx)

# # # load data (pointattN) UCD
# balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_ucd/"
# #balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_corrected_bbox/"
# #dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"
# load_idx = True
# limit = 150
# cld_completed = get_cloud_list_vcn(balanced_path, "pred", pred_confidence=True, limit=limit, load_idx=load_idx)
# cld_gt = get_cloud_list_vcn(balanced_path, "gt", pred_confidence=True, limit=limit, load_idx=load_idx)

# load data (pointattN) CD
balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_ucd/"
#balanced_path = "/home/haritha/documents/experiments/PointAttN/outputs_corrected_bbox/"
#dcd_path = "/home/haritha/documents/experiments/PointAttN/outputs_cd/"
load_idx = True
limit = 150
cld_completed = get_cloud_list_vcn(balanced_path, "pred", limit=limit, load_idx=load_idx)
cld_gt = get_cloud_list_vcn(balanced_path, "gt", limit=limit, load_idx=load_idx)

cld_completed_tensor = torch.tensor(cld_completed).cuda()
cld_gt_tensor = torch.tensor(cld_gt).cuda()


In [ ]:
# calculate pointwise chamfer distance point assignments
def calc_pointwise_chamfer_distance(pred, gt):
    chamferDist = ChamferDistance()
    nn = chamferDist(
        pred, gt, bidirectional=True, return_nn=True)
    return nn[0].idx, nn[1].idx

In [ ]:
assignment_0, assignment_1 = calc_pointwise_chamfer_distance(cld_completed_tensor, cld_gt_tensor)
assignment_0, assignment_1 = assignment_0.squeeze(), assignment_1.squeeze()
print(assignment_0.shape, assignment_1.shape)

In [ ]:
reverse_assignment = torch.gather(assignment_1, 1, assignment_0)

cuda = torch.device("cuda")
expected = torch.arange(assignment_1.shape[1], device=cuda)
expected = expected.repeat(assignment_1.shape[0], 1)

print(expected.shape, reverse_assignment.shape)

consistency = torch.sum(torch.eq(expected, reverse_assignment).long())/len(reverse_assignment)
print(consistency)